In [93]:
import collections

from Bio import pairwise2
from Bio.Seq import Seq
#import networkx as nx

from tf.app import use
A = use('etcbc/bhsa', hoist=globals())
Fmt, Tmt, Lmt = F, T, L

B = use('dt-ucph/sp', version='3.4', hoist=globals())
Fsp, Tsp, Lsp = F, T, L

C = use('etcbc/dss', version='1.9', hoist=globals())
Fdss, Tdss, Ldss = F, T, L

del F, T, L

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,5,79878.40,100
chapter,187,2135.79,100
verse,5841,68.38,100
word,114890,3.48,100
sign,399392,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [94]:
def align_verses(str_1, str_2):
        
    seq1 = Seq(str_1)
    seq2 = Seq(str_2)
    
    alignments = pairwise2.align.globalxx(seq1, seq2)
    
    seq1_al = (alignments[0][0]).strip(' ')
    seq2_al = (alignments[0][1]).strip(' ')
        
    return seq1_al, seq2_al

In [116]:
def prepare_book_data(F, T, book_name, verse_g_cons, word2char):
    for w in eval(F + ".otype.s('word')"):
        bo, ch, ve = eval(T + '.sectionFromNode(w)')
        if bo != book_name:
            continue
        
        g_cons = eval(F + '.g_cons.v(w)')
        trailer = eval(F + '.trailer.v(w)')
        if trailer:
            trailer = ' '
        verse_g_cons[(bo, ch, ve)].append(g_cons + trailer)
        for char in g_cons:
            word2char[(bo, ch, ve)].append(w)
        
    return verse_g_cons, word2char

In [179]:
def make_alignments(verse_text1, verse_text2):
    alignments_dict = {}

    for section, text1 in verse_text1.items():
        try:
            text2 = verse_text2[section]
            alignment1, alignment2 = align_verses(text1, text2)
            alignments_dict[section] = (alignment1, alignment2)
        except:
            continue
    return alignments_dict

In [181]:
def collect_matching_words(alignments_dict, word2char1, word2char2):
    man1_man2_dict = collections.defaultdict(list)

    for section, (al1, al2) in alignments_dict.items():
        man1_idx = 0
        man2_idx = 0
    
        word_chars1 = word2char1[section]
        word_chars2 = word2char2[section]
        for char1, char2 in zip(al1, al2):
            if char1 not in {' ', '-'}:
                man1_word = word_chars1[man1_idx]
                man1_idx += 1
            
            if char2 not in {' ', '-'}:
                man2_word = word_chars2[man2_idx]
                man2_idx += 1
            
            if char1 not in {' ', '-'} and char2 not in {' ', '-'}:
                man1_man2_dict[man1_word].append(man2_word)
                
    return man1_man2_dict

In [140]:
def most_frequent(List):
    return max(set(List), key = List.count)

# Match words in MT Numbers and SP NUmbers

In [147]:
mt_verse_g_cons = collections.defaultdict(list)
mt_word2char = collections.defaultdict(list)

verse_g_cons, word2char = prepare_book_data('Fmt', 'Tmt', 'Numbers', mt_verse_g_cons, mt_word2char)
mt_verse_text = {section : ''.join(g_conss).strip() for (section, g_conss) in mt_verse_g_cons.items()}

In [148]:
sp_verse_g_cons = collections.defaultdict(list)
sp_word2char = collections.defaultdict(list)

sp_verse_g_cons, sp_word2char = prepare_book_data('Fsp', 'Tsp', 'Numbers', sp_verse_g_cons, sp_word2char)
sp_verse_text = {section : ''.join(g_conss).strip() for (section, g_conss) in sp_verse_g_cons.items()}

In [180]:
mt_sp_alignments_dict = make_alignments(mt_verse_text, sp_verse_text)

In [138]:
matching_words_dict = collect_matching_words(mt_sp_alignments_dict, mt_word2char, sp_word2char)

In [197]:
mt_sp_matches = {}

for mt_word, sp_list in matching_words_dict.items():
    sp_word = most_frequent(sp_list)
    #print(mt_word, Fmt.g_cons.v(mt_word), sp_word, Fsp.g_cons.v(sp_word))
    mt_sp_matches[mt_word] = sp_word

In [198]:
mt_sp_matches

{77495: 1971406,
 77496: 1971407,
 77497: 1971408,
 77498: 1971409,
 77499: 1971410,
 77500: 1971411,
 77501: 1971412,
 77502: 1971413,
 77503: 1971414,
 77504: 1971416,
 77505: 1971417,
 77506: 1971419,
 77507: 1971420,
 77508: 1971421,
 77509: 1971422,
 77510: 1971423,
 77511: 1971424,
 77512: 1971425,
 77513: 1971426,
 77514: 1971427,
 77515: 1971428,
 77516: 1971429,
 77517: 1971430,
 77518: 1971431,
 77519: 1971433,
 77520: 1971434,
 77521: 1971435,
 77522: 1971436,
 77523: 1971437,
 77524: 1971438,
 77525: 1971439,
 77526: 1971440,
 77527: 1971441,
 77528: 1971442,
 77529: 1971443,
 77530: 1971444,
 77531: 1971445,
 77532: 1971446,
 77533: 1971447,
 77534: 1971448,
 77535: 1971449,
 77536: 1971450,
 77537: 1971451,
 77538: 1971452,
 77539: 1971454,
 77540: 1971455,
 77541: 1971456,
 77542: 1971457,
 77543: 1971458,
 77544: 1971459,
 77545: 1971460,
 77546: 1971461,
 77547: 1971462,
 77548: 1971463,
 77549: 1971464,
 77550: 1971465,
 77552: 1971466,
 77553: 1971467,
 77554: 197146

# Match words in MT Numbers and 4Q27 (= QSP scroll)

In [186]:

def prepare_dss_book_data(scroll_name, book_name, verse_g_cons, word2char):
    for scr in Fdss.otype.s('scroll'):
        if Tdss.scrollName(scr) == '4Q27':
            words = Ldss.d(scr, 'word')
            for w in words:
                bo = Fdss.book_etcbc.v(w)
                if bo != book_name:
                    continue
                ch = Fdss.chapter.v(w)
                ve = Fdss.verse.v(w)
                
                g_cons = Fdss.g_cons.v(w)
                after = Fdss.after.v(w)
                if after is None:
                    after = ''
                if g_cons:
                    verse_g_cons[(bo, int(ch), int(ve))].append(g_cons + after)
                    for char in g_cons:
                        word2char[(bo, int(ch), int(ve))].append(w)
    return verse_g_cons, word2char

In [187]:
dss_verse_g_cons = collections.defaultdict(list)
dss_word2char = collections.defaultdict(list)

verse_g_cons, word2char = prepare_dss_book_data('4Q27', 'Numbers', dss_verse_g_cons, dss_word2char)
dss_verse_text = {section : ''.join(g_conss).strip() for (section, g_conss) in dss_verse_g_cons.items()}


In [189]:
mt_dss_alignments_dict = make_alignments(mt_verse_text, dss_verse_text)

In [193]:
matching_words_mt_dss_dict = collect_matching_words(mt_dss_alignments_dict, mt_word2char, dss_word2char)

In [196]:
mt_dss_matches = {}

for mt_word, dss_list in matching_words_mt_dss_dict.items():
    dss_word = most_frequent(dss_list)
    mt_dss_matches[mt_word] = dss_word

In [ ]:
# Make Numbers network

In [ ]:
import pandas as pd

